# GAN exploration

In [3]:
# Class exercice
class polygon(): 
    def __init__(self, sides, name):
        self.sides = sides
        self.name = name

square = polygon(4, "Square")
pentagon = polygon(5, "Pentagon")
print(square.sides)

4
